In [18]:
import os 
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
import sklearn.metrics as sklm
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

C:\Users\AM7\#Work\Machine-Learning-Study-Groups


Read the dataset [ features ]

In [3]:
advworkscusts = pd.read_csv(dir_path + '\\Datasets\\Regression\\AdvWorksCusts.csv')
print(advworkscusts.shape)
advworkscusts.head()

(16519, 23)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,1966-04-08,Bachelors,Professional,M,M,1,0,0,2,137947
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,1965-05-14,Bachelors,Professional,M,S,0,1,3,3,101141
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,1965-08-12,Bachelors,Professional,M,M,1,1,3,3,91945
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,1968-02-15,Bachelors,Professional,F,S,0,1,0,0,86688
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,1968-08-08,Bachelors,Professional,F,S,1,4,5,5,92771


Read the dataset [ Labels ]

In [4]:
spend = pd.read_csv(dir_path + '\\Datasets\\Regression\\AW_AveMonthSpend.csv')
print(spend.shape)
spend.head()

(16519, 2)


,CustomerID,AveMonthSpend
0,11000,89
1,11001,117
2,11002,123
3,11003,50
4,11004,95


drop duplicates row

In [5]:
advworkscusts.drop_duplicates(subset = 'CustomerID', keep = 'first', inplace = True)
print(advworkscusts.shape)
print(advworkscusts.CustomerID.unique().shape)

spend.drop_duplicates(subset = 'CustomerID', keep = 'first', inplace = True)
print(spend.shape)
print(spend.CustomerID.unique().shape)

(16404, 23)
(16404,)
(16404, 2)
(16404,)


Here we merge the label column that name (AveMonthSpend) to the features dataset to be all thing in ne dataset or one table.

In [6]:
customerData = pd.merge(advworkscusts, spend)
print(customerData.shape)
customerData.head()


(16404, 24)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,Bachelors,Professional,M,M,1,0,0,2,137947,89
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,Bachelors,Professional,M,S,0,1,3,3,101141,117
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,Bachelors,Professional,M,M,1,1,3,3,91945,123
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,Bachelors,Professional,F,S,0,1,0,0,86688,50
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,Bachelors,Professional,F,S,1,4,5,5,92771,95


In this part, we convert BirthDate to age add assign it to his column.

In [7]:
from datetime import date

def calculate_age(bornyear, bornmonth, bornday):
    today = date.today()
    return today.year - bornyear - ((today.month, today.day) < (bornmonth, bornday))

In [8]:
t = []

for i in customerData['BirthDate']:
  birthday = str(i)
  birthday = birthday.split('-')
  b = calculate_age(int(birthday[0]), int(birthday[1]), int(birthday[2]))
  t.append(b)
  
customerData['BirthDate'] = t

We can see the edit now on BirthDate column.

In [9]:
 customerData['BirthDate']

0        52
1        53
2        53
3        50
4        50
5        53
6        52
7        54
8        54
9        54
10       54
11       55
12       50
13       50
14       50
15       39
16       39
17       40
18       40
19       40
20       40
21       72
22       72
23       71
24       72
25       71
26       71
27       71
28       71
29       70
         ..
16374    76
16375    39
16376    40
16377    40
16378    40
16379    40
16380    41
16381    41
16382    41
16383    39
16384    40
16385    41
16386    41
16387    75
16388    57
16389    57
16390    57
16391    58
16392    58
16393    58
16394    58
16395    59
16396    59
16397    59
16398    59
16399    59
16400    60
16401    58
16402    58
16403    59
Name: BirthDate, Length: 16404, dtype: int64

In [10]:
Feature_names = ['HomeOwnerFlag', 'BirthDate', 'NumberChildrenAtHome', 'TotalChildren', 'NumberCarsOwned','YearlyIncome'] ##numerical features
Features = np.array(customerData[Feature_names])
Labels = np.array(customerData['AveMonthSpend'])
print(len(Features), len(Labels))

16404 16404


Encode the string features to numerical features

In [11]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc_cat_features = enc.fit_transform(cat_features)
    
    
    
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit_transform(enc_cat_features.reshape(-1,1))
    return enc_cat_features.reshape(-1,1)

categorical_columns = ['Gender','MaritalStatus'] ## string features

for col in categorical_columns:
    temp = encode_string(advworkscusts[col])
    Features = np.concatenate([Features, temp], axis = 1) 

In [12]:
x_train, x_test, y_train, y_test = ms.train_test_split(Features, Labels, test_size=0.2)
print(len(x_train), len(x_test))

13123 3281


In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

C:\Users\AM7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Build Model

In [16]:
lin_mod = linear_model.LinearRegression()
lin_mod.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
print(lin_mod.intercept_) #b value
print(lin_mod.coef_) # m values

72.44113388706852
[-0.04880135  3.96736757 -1.68665565 16.5612872   0.45231932 -0.25002266
  3.96736757 14.09505915 -3.11418772]


In [20]:
y_predicted = lin_mod.predict(x_test)
r2 = sklm.r2_score(y_test, y_predicted)
print('R^2                    = ' + str(r2))

R^2                    = 0.9440379528311353
